In [1]:
import pandas as pd

# Load Store 1, Item 1 data (CSV you created from Excel)
df = pd.read_csv("../data/store1_item1_sales.csv")

# Convert date column to datetime
df["date"] = pd.to_datetime(df["date"], format="%d-%m-%Y")

# Sort by date
df = df.sort_values("date")

df.head()


,date,store,item,sales
0,2013-01-01,1,1,13
211816,2013-01-01,7,12,26
832656,2013-01-01,7,46,27
213642,2013-01-01,8,12,54
215468,2013-01-01,9,12,35


In [2]:
# Keep only Store = 1 and Item = 1
df = df[(df["store"] == 1) & (df["item"] == 1)]

# Reset index after filtering
df = df.reset_index(drop=True)

df.head(), df.tail(), len(df)


(        date  store  item  sales
 0 2013-01-01      1     1     13
 1 2013-01-02      1     1     11
 2 2013-01-03      1     1     14
 3 2013-01-04      1     1     13
 4 2013-01-05      1     1     10,
            date  store  item  sales
 1821 2017-12-27      1     1     14
 1822 2017-12-28      1     1     19
 1823 2017-12-29      1     1     15
 1824 2017-12-30      1     1     27
 1825 2017-12-31      1     1     23,
 1826)

In [3]:
# Keep only date and sales for forecasting
ts_df = df[["date", "sales"]].copy()

# Set date as index (time series format)
ts_df = ts_df.set_index("date")

ts_df.head(), ts_df.tail()


(            sales
 date             
 2013-01-01     13
 2013-01-02     11
 2013-01-03     14
 2013-01-04     13
 2013-01-05     10,
             sales
 date             
 2017-12-27     14
 2017-12-28     19
 2017-12-29     15
 2017-12-30     27
 2017-12-31     23)

In [4]:
# STEP 3: Create pure time-series data (date → sales)

# Keep only required columns
ts_df = df[["date", "sales"]].copy()

# Set date as index (VERY IMPORTANT)
ts_df.set_index("date", inplace=True)

# Ensure daily frequency
ts_df = ts_df.asfreq("D")

ts_df.head(), ts_df.tail()


(            sales
 date             
 2013-01-01     13
 2013-01-02     11
 2013-01-03     14
 2013-01-04     13
 2013-01-05     10,
             sales
 date             
 2017-12-27     14
 2017-12-28     19
 2017-12-29     15
 2017-12-30     27
 2017-12-31     23)

In [5]:
# STEP 4: Check missing values
ts_df.isna().sum()


sales    0
dtype: int64

In [6]:
# STEP 5: Train-Test Split (Time-based)

split_ratio = 0.8
split_index = int(len(ts_df) * split_ratio)

train_ts = ts_df.iloc[:split_index]
test_ts = ts_df.iloc[split_index:]

len(train_ts), len(test_ts)


(1460, 366)

In [7]:
# STEP 6.1: Naive Forecast

naive_forecast = [train_ts.iloc[-1].values[0]] * len(test_ts)

naive_forecast[:5]


[np.int64(24), np.int64(24), np.int64(24), np.int64(24), np.int64(24)]

In [8]:
from sklearn.metrics import mean_absolute_error

naive_mae = mean_absolute_error(test_ts, naive_forecast)
naive_mae


5.844262295081967

In [9]:
# STEP 6.3: Moving Average (7-day)

window = 7
moving_avg_value = train_ts[-window:].mean().values[0]
ma_forecast = [moving_avg_value] * len(test_ts)

ma_forecast[:5]


[np.float64(17.142857142857142),
 np.float64(17.142857142857142),
 np.float64(17.142857142857142),
 np.float64(17.142857142857142),
 np.float64(17.142857142857142)]

In [10]:
ma_mae = mean_absolute_error(test_ts, ma_forecast)
ma_mae


6.759953161592508

In [11]:
naive_mae, ma_mae


(5.844262295081967, 6.759953161592508)

In [12]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")


In [13]:
# Fit ARIMA model on training data
arima_model = ARIMA(train_ts, order=(5, 1, 0))
arima_fit = arima_model.fit()

arima_fit.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                  sales   No. Observations:                 1460
Model:                 ARIMA(5, 1, 0)   Log Likelihood               -4559.001
Date:                Sun, 08 Feb 2026   AIC                           9130.002
Time:                        18:25:11   BIC                           9161.715
Sample:                    01-01-2013   HQIC                          9141.833
                         - 12-30-2016                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.7228      0.025    -28.617      0.000      -0.772      -0.673
ar.L2         -0.5999      0.028    -21.461      0.000      -0.655      -0.545
ar.L3         -0.4933      0.030    -16.533      0.000      -0.552      -0.435
ar.L4         -0.3855      0.029    -13.407      0.000      -0.442      -0.329
ar.L5         -0.2449      0.024    -10.057      0.000      -0.293      -0.197
sigma2        30.2919      1.010     29.985      0.000      28.312      32.272
===================================================================================
Ljung-Box (L1) (Q):                  12.64   Jarque-Bera (JB):                19.79
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.44   Skew:                             0.13
Prob(H) (two-sided):                  0.00   Kurtosis:                         3.51
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [14]:
# Forecast for test period
arima_forecast = arima_fit.forecast(steps=len(test_ts))

arima_forecast[:5]


2016-12-31    19.164351
2017-01-01    17.549567
2017-01-02    16.741439
2017-01-03    18.299108
2017-01-04    19.584005
Freq: D, Name: predicted_mean, dtype: float64

In [15]:
from sklearn.metrics import mean_absolute_error

arima_mae = mean_absolute_error(test_ts, arima_forecast)
arima_mae


6.106221564642629

In [16]:
print(f"Naive MAE        : {naive_mae:.2f}")
print(f"Moving Avg MAE   : {ma_mae:.2f}")
print(f"ARIMA MAE        : {arima_mae:.2f}")


Naive MAE        : 5.84
Moving Avg MAE   : 6.76
ARIMA MAE        : 6.11


In [17]:
# Train ARIMA on FULL data
final_model = ARIMA(ts_df, order=(5,1,0))
final_fit = final_model.fit()

# Forecast next 30 days
future_steps = 30
future_forecast = final_fit.forecast(steps=future_steps)

future_forecast.head()


2018-01-01    19.248392
2018-01-02    18.709556
2018-01-03    19.766935
2018-01-04    19.785318
2018-01-05    21.797391
Freq: D, Name: predicted_mean, dtype: float64

In [18]:
future_df = future_forecast.reset_index()
future_df.columns = ["date", "forecast_sales"]

future_df.to_csv("../data/future_forecast.csv", index=False)


In [19]:
future_df.head()


,date,forecast_sales
0,2018-01-01,19.248392
1,2018-01-02,18.709556
2,2018-01-03,19.766935
3,2018-01-04,19.785318
4,2018-01-05,21.797391
